#Simple Chatbot

#Training the Chatbot


##Importing the libraries

In [35]:
import random 
import json
import pickle
import numpy as np
import nltk
import tensorflow


In [36]:
from tensorflow.keras.models import Sequential
from nltk.stem import WordNetLemmatizer   ##reducing to stem words
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD ##sgd (stochastic gradient descent)


##Reading the JSON file


In [37]:
intents = json.loads(open("intents3.json").read())

In [38]:
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi there', 'hi', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day', 'how is it doing?', 'hello chatbot', 'hi buddy!', "what's up?"], 'responses': ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?', 'hey!', 'hello', 'hi buddy', 'hi']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time', 'I am leaving', 'Bye Bye', 'See you Again', 'Ok bye', 'I was nice mwwt with you'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.', 'Sad to see you go :(', 'Talk to you later', 'Goodbye!', 'Bye buddy']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me', 'You are very helpful'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I

##Cleaning the dataset



In [39]:
words = []
classes = []
documents = []
ignore_letters = ["?","!",".",",","'"]

In [40]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [41]:
for intent in intents['intents']:
  for pattern in intent["patterns"]:
    word_list = nltk.word_tokenize(pattern)   ## "Hi there"-->> ["Hi","there"]
    words.extend(word_list)
    documents.append((word_list,intent["tag"]))
    if intent["tag"] not in classes:
      classes.append(intent["tag"])

In [42]:
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [43]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]

In [44]:
words = sorted(set(words))

In [45]:
classes = sorted(set(classes))
print(classes)

['beauty_products', 'company_and_assistant', 'goodbye', 'greeting', 'hours', 'options', 'thanks']


In [46]:
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl","wb"))

##Creating the bag of words model


In [47]:
training = []
output_empty = [0]*len(classes)

In [48]:
for document in documents:
  bag = []
  word_patterns = document[0]
  word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)
  output_row = list(output_empty)
  output_row[classes.index(document[1])] = 1
  training.append([bag, output_row])


In [49]:
random.shuffle(training)

In [50]:
training = np.array(training)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [51]:
train_X = list(training[:,0])
train_y = list(training[:,1])

##Creating the model

In [52]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),),activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation="softmax"))

In [53]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov = True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


###Fit the model

In [54]:
model.fit(np.array(train_X),np.array(train_y),epochs=200, batch_size=5, verbose=0)

###Save the model

In [55]:
model.save("ChatbotProject")
print("Done")



INFO:tensorflow:Assets written to: ChatbotProject/assets
Done


#Testing the Chatbot



##Importing The Libraries


In [1]:
import nltk
import json
import pickle
import random
import numpy as np
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model

##Loading the Data And Trained Chatbot Model


In [2]:
intents = json.loads(open("intents3.json").read())
words = pickle.load(open("words.pkl","rb"))
classes = pickle.load(open("classes.pkl","rb"))
model = load_model("ChatbotProject")

##Function for Creating responses of questions asked to chtbot

In [3]:
lemmatizer = WordNetLemmatizer()

In [4]:
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

In [5]:
def bag_of_words(sentence):

  sentence_words = clean_up_sentence(sentence)
  bag = [0]*len(words)
  for w in sentence_words:
    for i, word in enumerate(words):
      if word==w:
        bag[i] = 1
  return np.array(bag)

In [6]:
def predict_class(sentence):
  bow = bag_of_words(sentence)
  res = model.predict(np.array([bow]))[0]
  ERROR_THRESHOLD = 0.25

  results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
  results.sort(key=lambda x:x[1], reverse=True)
  
  return_list = []
  for r in results:
    return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
  return return_list

In [7]:

def response(sentence):
    results = predict_class(sentence)
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0]["intent"]:
                    print("Chatbot- ",random.choice(i['responses']))
                    if results[0]["intent"] =="goodbye":
                      return False
                    else: 
                      return True
            results.pop(0)

##Ask To Chatbot


In [9]:
answer = True
while answer:
    input_data = input("You- ")
    answer = response(input_data)
 
    

You- hello chatbot
Chatbot-  Hi there, how can I help?
You- How can you help me?
Chatbot-  I can tell you which products our company QueenBeauty offer
You- whats your name?
Chatbot-  hey, I am Beaut Assistant of Queen Beauty
You- what products your company offer?
Chatbot-  Beauty Cream, Skin Lightening Cream, Face Cream. Sunscreen, BB Cream, Skin Cleansing Cream, Barrier Cream and Collagen Cream are some of our products
You- what are your service hours?
Chatbot-  24/7
You- Happy to meet you chatbot
Chatbot-  See you!
